In [5]:
from subprocess import check_output
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
#import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [21]:
df = pd.read_csv('/Users/Suki/Desktop/Zeiss/New Customer.csv')

In [23]:
df_copy = pd.get_dummies(df)
df_copy.head()

,Customer_Purchase_History,Sales_Level,Purchase_Number,Salary,Marketing_Time,Alternative,Discount,Awareness,Marketing_Channel_Mail,Marketing_Channel_Online Advertisement,Marketing_Channel_Paid Search,Marketing_Channel_Personal Sales,Marketing_Channel_Referral,Marketing_Channel_Shop,Marketing_Channel_Social Media,Brand_Review_high,Brand_Review_low,Brand_Review_medium
0,0.12,1.00,3,2780,2.5,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.10,0.77,7,2500,3.0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.37,0.55,2,1270,2.0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.89,0.87,5,2550,3.0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.45,0.47,2,1350,2.0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
df1 = df_copy
y = df1['Awareness'].values
df1 = df1.drop(['Awareness'],axis=1)
X = df1.values

In [25]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.50)

In [26]:
log_reg = LogisticRegression()
log_reg.fit(Xtrain, ytrain)
y_val_l = log_reg.predict_proba(Xtest)
print("Validation accuracy: ", sum(pd.DataFrame(y_val_l).idxmax(axis=1).values
                                   == ytest)/len(ytest))

Validation accuracy:  0.834128215442


In [27]:
radm = RandomForestClassifier()
radm.fit(Xtrain, ytrain)
y_val_l = radm.predict_proba(Xtest)
print("Validation accuracy: ", sum(pd.DataFrame(y_val_l).idxmax(axis=1).values
                                   == ytest)/len(ytest))
clf = radm

Validation accuracy:  0.999961692429


In [28]:
indices = np.argsort(radm.feature_importances_)[::-1]

# Print the feature ranking
print('Feature ranking:')

for f in range(df1.shape[1]):
    print('%d. feature %d %s (%f)' % (f+1 , indices[f], df1.columns[indices[f]],
                                      radm.feature_importances_[indices[f]]))

Feature ranking:
1. feature 0 Customer_Purchase_History (0.272873)
2. feature 4 Marketing_Time (0.249233)
3. feature 3 Salary (0.150503)
4. feature 1 Sales_Level (0.150294)
5. feature 2 Purchase_Number (0.124381)
6. feature 5 Alternative (0.012803)
7. feature 15 Brand_Review_low (0.006784)
8. feature 14 Brand_Review_high (0.006187)
9. feature 11 Marketing_Channel_Referral (0.004223)
10. feature 9 Marketing_Channel_Paid Search (0.003679)
11. feature 16 Brand_Review_medium (0.003612)
12. feature 12 Marketing_Channel_Shop (0.003293)
13. feature 6 Discount (0.003153)
14. feature 10 Marketing_Channel_Personal Sales (0.003037)
15. feature 13 Marketing_Channel_Social Media (0.002617)
16. feature 7 Marketing_Channel_Mail (0.001753)
17. feature 8 Marketing_Channel_Online Advertisement (0.001575)


# The primary factors that influence customers to be aware of our products.

1. Customer_Purchase_History (0.27)
2. Marketing_Time (0.24)
3. Salary (0.15)
4. Sales_Level (0.15)
5. Purchase_Number (0.12)

# Predict who will be aware of our products soon.

In [29]:
stay = df[df['Awareness'] == 0]
stay_copy = pd.get_dummies(stay)

In [30]:
df1 = stay_copy
y = df1['Awareness'].values
df1 = df1.drop(['Awareness'],axis=1)
X = df1.values

In [31]:
pred = clf.predict_proba(X)

In [32]:
sum(pred[:,1]==1)

0

In [33]:
stay['Awareness Probability'] = pred[:,1]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Show who will likely to be aware of our products with probability greater than or equal to 95%.

In [37]:
stay[stay['Awareness Probability']>=0.95]

,Customer_Purchase_History,Sales_Level,Purchase_Number,Salary,Marketing_Time,Alternative,Discount,Marketing_Channel,Brand_Review,Awareness,Awareness Probability
94034,0.4,0.48,2,1550,2.0,0,0,Shop,low,0,0.955273
95709,0.4,0.48,2,1550,2.0,0,0,Shop,low,0,0.955273
97384,0.4,0.48,2,1550,2.0,0,0,Shop,low,0,0.955273
